In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import geopy
from geopy.geocoders import Nominatim

In [2]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

2023-08-09 16:22:56.794657: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
sample = pd.read_csv('data/submit_sample.csv')

In [4]:
train.head()

,id,region,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,state,price
0,0,nashville,1949,bmw,excellent,6 cylinders,gas,115148,clean,manual,rwd,mid-size,convertible,orange,NaN,27587
1,1,state college,2013,toyota,fair,8 cylinders,gas,172038,clean,automatic,rwd,full-size,sedan,silver,pa,4724
2,2,wichita,1998,ford,good,6 cylinders,gas,152492,clean,automatic,fwd,full-size,SUV,silver,ks,10931
3,3,albany,2014,ford,excellent,4 cylinders,gas,104118,clean,manual,fwd,mid-size,SUV,blue,ny,16553
4,4,redding,2005,ford,excellent,6 cylinders,gas,144554,clean,manual,fwd,mid-size,sedan,red,ca,5158


In [5]:
test.head()

,id,region,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,state
0,27532,western slope,2015,chevrolet,excellent,4 cylinders,gas,92553,clean,automatic,fwd,full-size,SUV,red,NaN
1,27533,roseburg,2013,nissan,like new,4 cylinders,gas,134385,salvage,automatic,fwd,mid-size,sedan,black,or
2,27534,akron / canton,2011,volkswagen,good,4 cylinders,gas,102489,clean,automatic,fwd,full-size,sedan,black,oh
3,27535,denver,2016,jeep,excellent,6 cylinders,diesel,64310,clean,automatic,4wd,mid-size,SUV,red,co
4,27536,hickory / lenoir,1999,honda,excellent,8 cylinders,gas,180839,rebuilt,automatic,4wd,mid-size,SUV,silver,nc


In [6]:
print(train.shape)
print(test.shape)

(27532, 16)
(27537, 15)


In [ ]:
# Create a geolocator object
headers = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'
geolocator = Nominatim(user_agent=headers)

# Define a function to get latitude and longitude for a given region
def get_lat_long(region):
    try:
        location = geolocator.geocode(region)
        if location:
            return location.latitude, location.longitude
        else:
            return None, None
    except:
        try:
            location = geolocator.geocode(region)
            if location:
                return location.latitude, location.longitude
            else:
                return None, None
        except:
            return None, None

In [ ]:
import time
loca = []
lat = []
lon = []
for location in set(train['region']):
    latitude, longitude = get_lat_long(location)
    loca.append(location)
    lat.append(latitude)
    lon.append(longitude)
    time.sleep(1)

In [6]:
lat_long_df = pd.DataFrame()
lat_long_df['region'] = loca
lat_long_df['latitude'] = lat
lat_long_df['longitude'] = lon
lat_long_df.head()

NameError: name 'loca' is not defined

In [ ]:
lat_long_df.isnull().sum()

region       0
latitude     0
longitude    0
dtype: int64

In [ ]:
lat_long_df.to_csv('lat_long_info.csv', index = False)

In [7]:
lat_long_df = pd.read_csv('lat_long_info.csv')

In [8]:
train_prep = train.merge(lat_long_df, on = 'region', how = 'left')
train_prep = train_prep.drop(['region', 'state'], axis = 1)

In [9]:
def preprocess_text(text):
    # Full-width ASCII characters mapping
    full_width_ascii_mapping = {
        '０': '0', '１': '1', '２': '2', '３': '3', '４': '4',
        '５': '5', '６': '6', '７': '7', '８': '8', '９': '9',
        'Ａ': 'A', 'Ｂ': 'B', 'Ｃ': 'C', 'Ｄ': 'D', 'Ｅ': 'E',
        'Ｆ': 'F', 'Ｇ': 'G', 'Ｈ': 'H', 'Ｉ': 'I', 'Ｊ': 'J',
        'Ｋ': 'K', 'Ｌ': 'L', 'Ｍ': 'M', 'Ｎ': 'N', 'Ｏ': 'O',
        'Ｐ': 'P', 'Ｑ': 'Q', 'Ｒ': 'R', 'Ｓ': 'S', 'Ｔ': 'T',
        'Ｕ': 'U', 'Ｖ': 'V', 'Ｗ': 'W', 'Ｘ': 'X', 'Ｙ': 'Y',
        'Ｚ': 'Z', 'ａ': 'a', 'ｂ': 'b', 'ｃ': 'c', 'ｄ': 'd',
        'ｅ': 'e', 'ｆ': 'f', 'ｇ': 'g', 'ｈ': 'h', 'ｉ': 'i',
        'ｊ': 'j', 'ｋ': 'k', 'ｌ': 'l', 'ｍ': 'm', 'ｎ': 'n',
        'ｏ': 'o', 'ｐ': 'p', 'ｑ': 'q', 'ｒ': 'r', 'ｓ': 's',
        'ｔ': 't', 'ｕ': 'u', 'ｖ': 'v', 'ｗ': 'w', 'ｘ': 'x',
        'ｙ': 'y', 'ｚ': 'z', 'ᴄ': 'c', '−': '-', 'ー': '-',
        'α': 'a'
    }

    # Replace full-width ASCII characters with their corresponding ASCII characters
    for fw_char, ascii_char in full_width_ascii_mapping.items():
        text = text.replace(fw_char, ascii_char)

    # Remove other full-width spaces (U+3000) and regular spaces
    text = text.replace('\u3000', '').replace(' ', '').replace('-', '').lower().strip()

    return text

In [10]:
train_prep['car_age'] = 2023 - train_prep['year']

In [11]:
train_prep['cylinders'] = train_prep['cylinders'].str.extract(r'(\d+)').astype(float).fillna(-1)

In [12]:
train_prep['size'] = train_prep['size'].apply(preprocess_text)

In [13]:
train_prep['size'] = train_prep['size'].apply(lambda x: 'compact' if x == 'sub-compact' else x)

In [14]:
#train_prep['market_segment'] = train_prep['type'] + '_' + train_prep['size']

In [15]:
train_prep['automatic_transmission'] = train_prep['transmission'].apply(lambda x: 1 if x.lower() == 'automatic' else 0)
train_prep['manual_transmission'] = train_prep['transmission'].apply(lambda x: 1 if x.lower() == 'manual' else 0)

In [16]:
condition_mapping = {'excellent': 5, 'like new': 4, 'new': 3, 'good':2, 'fair': 1, 'salvage':0}
train_prep['condition'] = train_prep['condition'].map(condition_mapping)

In [17]:
train_prep['gas_fueled'] = train_prep['fuel'].apply(lambda x: 1 if x == 'gas' else 0)
train_prep['diesel_fueled'] = train_prep['fuel'].apply(lambda x: 1 if x == 'diesel' else 0)
train_prep['hybrid_fueled'] = train_prep['fuel'].apply(lambda x: 1 if x == 'hybrid' else 0)
train_prep['electric_fueled'] = train_prep['fuel'].apply(lambda x: 1 if x == 'electric' else 0)

In [18]:
train_prep['front_wheel_drive'] = train_prep['drive'].apply(lambda x: 1 if x == 'fwd' else 0)
train_prep['four_wheel_drive'] = train_prep['drive'].apply(lambda x: 1 if x == '4wd' else 0)

In [19]:
def generate_embeddings(df, cat_col, name):
    df[cat_col].fillna('other', inplace=True)
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(df[cat_col])
    sequences = tokenizer.texts_to_sequences(df[cat_col])

    max_sequence_length = max(len(seq) for seq in sequences)
    padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)
    
    vocab_size = len(tokenizer.word_index) + 1
    
    num_cat = df[cat_col].nunique()
    embedding_size = min(50, int((num_cat+1)/ 2))
    
    model = Sequential()
    model.add(Embedding(input_dim = vocab_size, output_dim = embedding_size, input_length = max_sequence_length, name="embedding"))
    model.compile(loss="mse", optimizer="adam", metrics=["accuracy"])
    model.fit(x=padded_sequences, y=padded_sequences, epochs=50, batch_size=16, verbose=0)
    
    embedding_layer = model.layers[0]
    learned_embeddings = embedding_layer.get_weights()[0] 

    new_df = pd.DataFrame()
    new_df[cat_col] = df[cat_col]
    
    embeddings = {}
    for word, index in tokenizer.word_index.items():
        embeddings[word] = learned_embeddings[index]

    embedding_dimensions = learned_embeddings.shape[1]
    for i in range(embedding_dimensions):
        embedding_column = f'{name}_{i+1}'
        new_df[embedding_column] = new_df[cat_col].map(lambda x: embeddings[x][i])
    
    return new_df.drop(cat_col, axis=1)

In [20]:
paint_embeddings = generate_embeddings(train_prep, 'paint_color', 'paint_embeddings')

2023-08-09 16:23:29.539105: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-08-09 16:23:29.539152: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-08-09 16:23:29.539186: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (default): /proc/driver/nvidia/version does not exist
2023-08-09 16:23:29.539397: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [21]:
paint_embeddings.head()

,paint_embeddings_1,paint_embeddings_2,paint_embeddings_3,paint_embeddings_4,paint_embeddings_5,paint_embeddings_6
0,3.332631,3.332631,3.332631,3.332631,3.332631,3.332631
1,2.854549,2.854549,2.854549,2.854549,2.854549,2.854549
2,2.854549,2.854549,2.854549,2.854549,2.854549,2.854549
3,2.907782,2.907782,2.907782,2.907782,2.907782,2.907782
4,3.090197,3.090197,3.090197,3.090197,3.090197,3.090197


In [22]:
train_prep = pd.concat([train_prep, paint_embeddings], axis =1)

In [23]:
train_prep['manufacturer'] = train_prep['manufacturer'].apply(preprocess_text)
col_to_keep = []
for brand, values in dict(train_prep['manufacturer'].value_counts()).items():
    if values >= 150:
        col_to_keep.append(brand)
train_prep['manufacturer'] = train_prep['manufacturer'].apply(lambda x: 'other' if x not in col_to_keep else x)

In [24]:
brand_embeddings = generate_embeddings(train_prep, 'manufacturer', 'brand_embeddings')

In [25]:
brand_embeddings.head()

,brand_embeddings_1,brand_embeddings_2,brand_embeddings_3,brand_embeddings_4,brand_embeddings_5,brand_embeddings_6,brand_embeddings_7,brand_embeddings_8,brand_embeddings_9,brand_embeddings_10,brand_embeddings_11,brand_embeddings_12,brand_embeddings_13
0,5.667300,5.667300,5.667300,5.667300,5.667300,5.667300,5.667300,5.667300,5.667300,5.667300,5.667300,5.667300,5.667300
1,5.683877,5.683877,5.683877,5.683877,5.683877,5.683877,5.683877,5.683877,5.683877,5.683877,5.683877,5.683877,5.683877
2,5.506867,5.506867,5.506867,5.506867,5.506867,5.506867,5.506867,5.506867,5.506867,5.506867,5.506867,5.506867,5.506867
3,5.506867,5.506867,5.506867,5.506867,5.506867,5.506867,5.506867,5.506867,5.506867,5.506867,5.506867,5.506867,5.506867
4,5.506867,5.506867,5.506867,5.506867,5.506867,5.506867,5.506867,5.506867,5.506867,5.506867,5.506867,5.506867,5.506867


In [26]:
train_prep = pd.concat([train_prep, brand_embeddings], axis =1)

In [27]:
train_prep['type'].fillna('other', inplace = True)
train_prep['type'] = train_prep['type'].apply(preprocess_text)

In [ ]:
type_embeddings = generate_embeddings(train_prep, 'type', 'type_embeddings')
size_embeddings = generate_embeddings(train_prep, 'size', 'size_embeddings')

In [ ]:
train_prep = pd.concat([train_prep, type_embeddings, size_embeddings], axis =1)

In [31]:
train_prep['title_status_new'] = train['title_status']

In [32]:
train_prep['title_status_new'].value_counts()

title_status_new
clean         22365
rebuilt        3004
salvage         985
lien            465
parts only      133
missing         124
Name: count, dtype: int64

In [33]:
train_prep['title_status_new'].fillna('clean', inplace = True)

In [34]:
train_prep['title_status_new'] = train_prep['title_status_new'].apply(preprocess_text)
status_embeddings = generate_embeddings(train_prep, 'title_status_new', 'title_status_embeddings')
train_prep = pd.concat([train_prep, status_embeddings], axis =1)

In [35]:
train_prep = train_prep.drop('title_status_new', axis = 1)

In [36]:
train_prep

,id,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,...,type_embeddings_3,type_embeddings_4,type_embeddings_5,type_embeddings_6,type_embeddings_7,size_embeddings_1,size_embeddings_2,title_status_embeddings_1,title_status_embeddings_2,title_status_embeddings_3
0,0,1949,bmw,5,6.0,gas,115148,clean,manual,rwd,...,3.207522,3.207522,3.207522,3.207522,3.207522,1.632407,1.632407,1.267552,1.267552,1.267552
1,1,2013,toyota,1,8.0,gas,172038,clean,automatic,rwd,...,2.668944,2.668944,2.668944,2.668944,2.668944,1.558996,1.558996,1.267552,1.267552,1.267552
2,2,1998,ford,2,6.0,gas,152492,clean,automatic,fwd,...,2.747221,2.747221,2.747221,2.747221,2.747221,1.558996,1.558996,1.267552,1.267552,1.267552
3,3,2014,ford,5,4.0,gas,104118,clean,manual,fwd,...,2.747221,2.747221,2.747221,2.747221,2.747221,1.632407,1.632407,1.267552,1.267552,1.267552
4,4,2005,ford,5,6.0,gas,144554,clean,manual,fwd,...,2.668944,2.668944,2.668944,2.668944,2.668944,1.632407,1.632407,1.267552,1.267552,1.267552
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27527,27527,2008,ford,2,6.0,gas,26660,clean,automatic,rwd,...,2.801944,2.801944,2.801944,2.801944,2.801944,1.695574,1.695574,1.267552,1.267552,1.267552
27528,27528,2007,ford,5,8.0,gas,108072,clean,automatic,rwd,...,2.914915,2.914915,2.914915,2.914915,2.914915,1.558996,1.558996,1.267552,1.267552,1.267552
27529,27529,2019,jeep,4,6.0,gas,139908,clean,automatic,4wd,...,2.747221,2.747221,2.747221,2.747221,2.747221,1.632407,1.632407,1.267552,1.267552,1.267552
27530,27530,2007,jeep,5,6.0,gas,112326,clean,automatic,4wd,...,2.668944,2.668944,2.668944,2.668944,2.668944,1.632407,1.632407,1.267552,1.267552,1.267552


In [ ]:
fuel_embeddings = generate_embeddings(train_prep, 'fuel', 'fuel_embeddings')
transmission_embeddings = generate_embeddings(train_prep, 'transmission', 'transmission_embeddings')
drive_embeddings = generate_embeddings(train_prep, 'drive', 'drive_embeddings')

In [ ]:
train_prep = pd.concat([train_prep, fuel_embeddings, fuel_embeddings, transmission_embeddings], axis =1)

In [ ]:
train_prep['odometer_log'] = np.log1p(train_prep['odometer'])

/home/studio-lab-user/.conda/envs/tf-gpu-310/lib/python3.10/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/studio-lab-user/.conda/envs/tf-gpu-310/lib/python3.10/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [ ]:
train_prep

,id,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,...,title_status_embeddings_3,fuel_embeddings_1,fuel_embeddings_2,fuel_embeddings_3,fuel_embeddings_1,fuel_embeddings_2,fuel_embeddings_3,transmission_embeddings_1,transmission_embeddings_2,odometer_log
0,0,1949,bmw,5,6.0,gas,115148,clean,manual,rwd,...,1.267552,1.184876,1.184876,1.184876,1.184876,1.184876,1.184876,1.334532,1.334532,11.653982
1,1,2013,toyota,1,8.0,gas,172038,clean,automatic,rwd,...,1.267552,1.184876,1.184876,1.184876,1.184876,1.184876,1.184876,1.275842,1.275842,12.055476
2,2,1998,ford,2,6.0,gas,152492,clean,automatic,fwd,...,1.267552,1.184876,1.184876,1.184876,1.184876,1.184876,1.184876,1.275842,1.275842,11.934874
3,3,2014,ford,5,4.0,gas,104118,clean,manual,fwd,...,1.267552,1.184876,1.184876,1.184876,1.184876,1.184876,1.184876,1.334532,1.334532,11.553290
4,4,2005,ford,5,6.0,gas,144554,clean,manual,fwd,...,1.267552,1.184876,1.184876,1.184876,1.184876,1.184876,1.184876,1.334532,1.334532,11.881415
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27527,27527,2008,ford,2,6.0,gas,26660,clean,automatic,rwd,...,1.267552,1.184876,1.184876,1.184876,1.184876,1.184876,1.184876,1.275842,1.275842,10.190957
27528,27528,2007,ford,5,8.0,gas,108072,clean,automatic,rwd,...,1.267552,1.184876,1.184876,1.184876,1.184876,1.184876,1.184876,1.275842,1.275842,11.590562
27529,27529,2019,jeep,4,6.0,gas,139908,clean,automatic,4wd,...,1.267552,1.184876,1.184876,1.184876,1.184876,1.184876,1.184876,1.275842,1.275842,11.848747
27530,27530,2007,jeep,5,6.0,gas,112326,clean,automatic,4wd,...,1.267552,1.184876,1.184876,1.184876,1.184876,1.184876,1.184876,1.275842,1.275842,11.629170


In [41]:
train_prep['title_status'].value_counts()

title_status
clean         22365
rebuilt        3004
salvage         985
lien            465
parts only      133
missing         124
Name: count, dtype: int64

In [53]:
train_prep['title_status'] = train['title_status']

In [54]:
train_prep['title_status'].fillna('clean', inplace = True)

In [55]:
status_mapping = {'clean': 5, 'rebuilt': 4, 'parts only' : 3, 'salvage':2, 'lien': 1, 'missing':0}
train_prep['title_status'] = train_prep['title_status'].map(status_mapping)

In [56]:
train_prep['title_status'].isnull().sum()

0

In [59]:
train_prep

,id,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,...,title_status_embeddings_3,fuel_embeddings_1,fuel_embeddings_2,fuel_embeddings_3,fuel_embeddings_1,fuel_embeddings_2,fuel_embeddings_3,transmission_embeddings_1,transmission_embeddings_2,odometer_log
0,0,1949,bmw,5,6.0,gas,115148,5,manual,rwd,...,1.267552,1.184876,1.184876,1.184876,1.184876,1.184876,1.184876,1.334532,1.334532,11.653982
1,1,2013,toyota,1,8.0,gas,172038,5,automatic,rwd,...,1.267552,1.184876,1.184876,1.184876,1.184876,1.184876,1.184876,1.275842,1.275842,12.055476
2,2,1998,ford,2,6.0,gas,152492,5,automatic,fwd,...,1.267552,1.184876,1.184876,1.184876,1.184876,1.184876,1.184876,1.275842,1.275842,11.934874
3,3,2014,ford,5,4.0,gas,104118,5,manual,fwd,...,1.267552,1.184876,1.184876,1.184876,1.184876,1.184876,1.184876,1.334532,1.334532,11.553290
4,4,2005,ford,5,6.0,gas,144554,5,manual,fwd,...,1.267552,1.184876,1.184876,1.184876,1.184876,1.184876,1.184876,1.334532,1.334532,11.881415
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27527,27527,2008,ford,2,6.0,gas,26660,5,automatic,rwd,...,1.267552,1.184876,1.184876,1.184876,1.184876,1.184876,1.184876,1.275842,1.275842,10.190957
27528,27528,2007,ford,5,8.0,gas,108072,5,automatic,rwd,...,1.267552,1.184876,1.184876,1.184876,1.184876,1.184876,1.184876,1.275842,1.275842,11.590562
27529,27529,2019,jeep,4,6.0,gas,139908,5,automatic,4wd,...,1.267552,1.184876,1.184876,1.184876,1.184876,1.184876,1.184876,1.275842,1.275842,11.848747
27530,27530,2007,jeep,5,6.0,gas,112326,5,automatic,4wd,...,1.267552,1.184876,1.184876,1.184876,1.184876,1.184876,1.184876,1.275842,1.275842,11.629170


In [ ]:


def data_wrangler(df: pd.DataFrame):
    df_prep = df.drop(['id','region','state'], axis=1)
    df_prep['manufacturer'] = df_prep['manufacturer'].apply(preprocess_text)
    col_to_keep = []
    for brand, values in dict(df_prep['manufacturer'].value_counts()).items():
        if values >= 1000:
            col_to_keep.append(brand)
    df_prep['manufacturer'] = df_prep['manufacturer'].apply(lambda x: 'other' if x not in col_to_keep else x)
    df_prep['size'] = df_prep['size'].apply(preprocess_text)
    df_prep['cylinders'] = df_prep['cylinders'].str.extract(r'(\d+)').astype(float).fillna(0)
    df_prep['condition'] = df_prep['condition'].apply(lambda x: 'fair' if x == 'salvage' else 'new' if x == 'like new' else x)
    condition_encoder = LabelEncoder()
    df_prep['condition'] = condition_encoder.fit_transform(df_prep['condition'])
    cols_to_ohe = ['manufacturer', 'fuel', 'title_status', 'transmission', 'drive', 'size', 'type', 'paint_color']
    df_prep = pd.get_dummies(df_prep, columns=cols_to_ohe)
    useless_cols = ['manufacturer_other', 'fuel_other', 'title_status_parts only', 'transmission_other',
                'drive_rwd', 'size_sub-compact', 'type_other', 'paint_color_purple']

    return df_prep.drop(columns=useless_cols, axis=1)

In [ ]:
train_prep = data_wrangler(train)
test_prep = data_wrangler(test)

In [ ]:
print(train_prep.shape)
print(test_prep.shape)

(27532, 52)
(27537, 51)


In [ ]:
! pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 8.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_predict, KFold, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
import lightgbm as lgb
import catboost as cb

def mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def mape_scorer(estimator, X, y_true):
    y_pred = estimator.predict(X)
    return mape(y_true, y_pred)

def baseline_models_cv(X, y):
    # Define the baseline models
    models = {
        'Random Forest': RandomForestRegressor(random_state=42),
        'XGBoost': xgb.XGBRegressor(random_state=42),
        'LightGBM': lgb.LGBMRegressor(random_state=42),
        'CatBoost': cb.CatBoostRegressor(random_state=42, verbose=0)
    }

    # Perform 5-fold cross-validation and compute MAPE for each model
    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    results = {}
    for model_name, model in models.items():
        scores = cross_val_predict(model, X, y, cv=cv, method='predict', n_jobs=-1, verbose=0)
        mape_score = mape(y, scores)
        results[model_name] = mape_score

    return results

In [ ]:
X = train_prep.drop('price', axis = 1)
y = train_prep['price']

In [ ]:
cv_results = baseline_models_cv(X, y)

# Display the Mean and Standard Deviation of MAPE for each model
for model_name, scores in cv_results.items():
    print(f"{model_name}: MAPE Mean = {scores.mean():.2f}%")

Random Forest: MAPE Mean = 70.15%
XGBoost: MAPE Mean = 67.15%
LightGBM: MAPE Mean = 65.98%
CatBoost: MAPE Mean = 65.82%


Best performing model is CatBoost so we will further tune it to give us the best results.

In [ ]:
catboost_model = cb.CatBoostRegressor(random_state=42, verbose=0)

# Define the hyperparameter grid to search over
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],
    'depth': [6, 8, 10],
    'l2_leaf_reg': [1, 3, 5],
    'iterations': [100, 200, 300]
}

# Perform 5-fold cross-validation with hyperparameter tuning
cv = KFold(n_splits=5, shuffle=True, random_state=42)
grid_search = GridSearchCV(catboost_model, param_grid, cv=cv, scoring='neg_mean_absolute_error', n_jobs=-1)
grid_search.fit(X, y)

# Get the best hyperparameters and the corresponding model
best_params = grid_search.best_params_
best_catboost_model = grid_search.best_estimator_

# Perform 5-fold cross-validation with the best model
cv_results = cross_val_score(best_catboost_model, X, y, cv=cv, scoring='neg_mean_absolute_error')

# Calculate the mean and standard deviation of the negative mean absolute error (MAE)
mean_mae = -cv_results.mean()
std_mae = cv_results.std()

# Calculate the mean and standard deviation of the Mean Absolute Percentage Error (MAPE)
mean_mape = mape(y, best_catboost_model.predict(X)).mean()
std_mape = mape(y, best_catboost_model.predict(X)).std()

# Display the results
print("Best Hyperparameters:", best_params)
print(f"MAE (Mean ± Std): {mean_mae:.2f} ± {std_mae:.2f}")
print(f"MAPE (Mean ± Std): {mean_mape:.2f}% ± {std_mape:.2f}%")

Best Hyperparameters: {'depth': 8, 'iterations': 300, 'l2_leaf_reg': 5, 'learning_rate': 0.05}
MAE (Mean ± Std): 5802.35 ± 57.38
MAPE (Mean ± Std): 63.06% ± 0.00%


In [ ]:
catboost_model = cb.CatBoostRegressor(random_state=42, depth=8, iterations=500000, l2_leaf_reg=5, verbose=5000)

In [ ]:
catboost_model.fit(X,y)

0:	learn: 10838.1036067	total: 9.42ms	remaining: 1h 18m 29s
5000:	learn: 5907.0404319	total: 42.4s	remaining: 1h 10m
10000:	learn: 4925.6470938	total: 1m 26s	remaining: 1h 10m 49s
15000:	learn: 4267.8248819	total: 2m 9s	remaining: 1h 9m 36s
20000:	learn: 3782.7556751	total: 2m 51s	remaining: 1h 8m 25s
25000:	learn: 3379.5956175	total: 3m 33s	remaining: 1h 7m 37s
30000:	learn: 3051.1348095	total: 4m 17s	remaining: 1h 7m 17s
35000:	learn: 2775.2932574	total: 4m 59s	remaining: 1h 6m 22s
40000:	learn: 2538.7702783	total: 5m 41s	remaining: 1h 5m 28s
45000:	learn: 2336.9687941	total: 6m 23s	remaining: 1h 4m 36s
50000:	learn: 2160.4544730	total: 7m 7s	remaining: 1h 4m 7s
55000:	learn: 2002.3457807	total: 7m 49s	remaining: 1h 3m 18s
60000:	learn: 1860.5597569	total: 8m 31s	remaining: 1h 2m 30s
65000:	learn: 1736.6689062	total: 9m 13s	remaining: 1h 1m 42s
70000:	learn: 1623.2496203	total: 9m 57s	remaining: 1h 1m 10s
75000:	learn: 1519.3361561	total: 10m 40s	remaining: 1h 27s
80000:	learn: 1428.

In [ ]:
ypred = catboost_model.predict(X)

In [ ]:
mape(y,ypred)

0.2790259374078595

In [ ]:
sample

,27532,14994.540583136328
0,27533,10004.210369
1,27534,8000.623545
2,27535,15062.223593
3,27536,8994.715270
4,27537,14994.540583
...,...,...
27531,55064,16435.629730
27532,55065,10004.210369
27533,55066,13968.291393
27534,55067,13968.291393


import numpy as np
from google.colab import autoviz
df_232280417079789223 = autoviz.get_df('df_232280417079789223')

def value_plot(df, y, sort_ascending=False, figsize=(2, 1)):
  from matplotlib import pyplot as plt
  if sort_ascending:
    df = df.sort_values(y).reset_index(drop=True)
  _, ax = plt.subplots(figsize=figsize)
  df[y].plot(kind='line')
  plt.title(y)
  ax.spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(df_232280417079789223, *['27532'], **{})
chart

import numpy as np
from google.colab import autoviz
df_232280417079789223 = autoviz.get_df('df_232280417079789223')

def value_plot(df, y, sort_ascending=False, figsize=(2, 1)):
  from matplotlib import pyplot as plt
  if sort_ascending:
    df = df.sort_values(y).reset_index(drop=True)
  _, ax = plt.subplots(figsize=figsize)
  df[y].plot(kind='line')
  plt.title(y)
  ax.spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(df_232280417079789223, *['14994.540583136328'], **{})
chart

import numpy as np
from google.colab import autoviz
df_232280417079789223 = autoviz.get_df('df_232280417079789223')

def histogram(df, colname, num_bins=20, figsize=(2, 1)):
  from matplotlib import pyplot as plt
  _, ax = plt.subplots(figsize=figsize)
  plt.hist(df[colname], bins=num_bins, histtype='stepfilled')
  plt.ylabel('count')
  plt.title(colname)
  ax.spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(df_232280417079789223, *['27532'], **{})
chart

import numpy as np
from google.colab import autoviz
df_232280417079789223 = autoviz.get_df('df_232280417079789223')

def histogram(df, colname, num_bins=20, figsize=(2, 1)):
  from matplotlib import pyplot as plt
  _, ax = plt.subplots(figsize=figsize)
  plt.hist(df[colname], bins=num_bins, histtype='stepfilled')
  plt.ylabel('count')
  plt.title(colname)
  ax.spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(df_232280417079789223, *['14994.540583136328'], **{})
chart

import numpy as np
from google.colab import autoviz
df_232280417079789223 = autoviz.get_df('df_232280417079789223')

def scatter_plots(df, colname_pairs, scatter_plot_size=2.5, size=8, alpha=.6):
  from matplotlib import pyplot as plt
  plt.figure(figsize=(len(colname_pairs) * scatter_plot_size, scatter_plot_size))
  for plot_i, (x_colname, y_colname) in enumerate(colname_pairs, start=1):
    ax = plt.subplot(1, len(colname_pairs), plot_i)
    ax.scatter(df[x_colname], df[y_colname], s=size, alpha=alpha)
    plt.xlabel(x_colname)
    plt.ylabel(y_colname)
    ax.spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = scatter_plots(df_232280417079789223, *[[['27532', '14994.540583136328']]], **{})
chart

import numpy as np
from google.colab import autoviz
df_232280417079789223 = autoviz.get_df('df_232280417079789223')

def time_series_multiline(df, timelike_colname, value_colname, series_colname, figsize=(2.5, 1.3), mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  palette = list(sns.palettes.mpl_palette(mpl_palette_name))
  def _plot_series(series, series_name, series_index=0):
    if value_colname == 'count()':
      counted = (series[timelike_colname]
                 .value_counts()
                 .reset_index(name='counts')
                 .rename({'index': timelike_colname}, axis=1)
                 .sort_values(timelike_colname, ascending=True))
      xs = counted[timelike_colname]
      ys = counted['counts']
    else:
      xs = series[timelike_colname]
      ys = series[value_colname]
    plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

  fig, ax = plt.subplots(figsize=figsize, layout='constrained')
  df = df.sort_values(timelike_colname, ascending=True)
  if series_colname:
    for i, (series_name, series) in enumerate(df.groupby(series_colname)):
      _plot_series(series, series_name, i)
    fig.legend(title=series_colname, bbox_to_anchor=(1, 1), loc='upper left')
  else:
    _plot_series(df, '')
  sns.despine(fig=fig, ax=ax)
  plt.xlabel(timelike_colname)
  plt.ylabel(value_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = time_series_multiline(df_232280417079789223, *['27532', '14994.540583136328', None], **{})
chart

import numpy as np
from google.colab import autoviz
df_232280417079789223 = autoviz.get_df('df_232280417079789223')

def time_series_multiline(df, timelike_colname, value_colname, series_colname, figsize=(2.5, 1.3), mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  palette = list(sns.palettes.mpl_palette(mpl_palette_name))
  def _plot_series(series, series_name, series_index=0):
    if value_colname == 'count()':
      counted = (series[timelike_colname]
                 .value_counts()
                 .reset_index(name='counts')
                 .rename({'index': timelike_colname}, axis=1)
                 .sort_values(timelike_colname, ascending=True))
      xs = counted[timelike_colname]
      ys = counted['counts']
    else:
      xs = series[timelike_colname]
      ys = series[value_colname]
    plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

  fig, ax = plt.subplots(figsize=figsize, layout='constrained')
  df = df.sort_values(timelike_colname, ascending=True)
  if series_colname:
    for i, (series_name, series) in enumerate(df.groupby(series_colname)):
      _plot_series(series, series_name, i)
    fig.legend(title=series_colname, bbox_to_anchor=(1, 1), loc='upper left')
  else:
    _plot_series(df, '')
  sns.despine(fig=fig, ax=ax)
  plt.xlabel(timelike_colname)
  plt.ylabel(value_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = time_series_multiline(df_232280417079789223, *['27532', 'count()', None], **{})
chart

In [ ]:
submission = catboost_model.predict(test_prep)

In [ ]:
columns = ['27532', '9560.750306145466']

In [ ]:
submission[0]

9560.750306145466

In [ ]:
sub = pd.DataFrame()
sub['27532'] = test['id'][1:]
sub['9560.750306145466'] = submission[1:]

In [ ]:
sub

,27532,9560.750306145466
1,27533,4979.631681
2,27534,7237.407326
3,27535,43815.644811
4,27536,17367.592884
5,27537,7616.022125
...,...,...
27532,55064,21046.033336
27533,55065,24007.871213
27534,55066,13297.289329
27535,55067,9131.548806


In [ ]:
sub.to_csv('sub_signate_2.csv', index = False)